## Import Python libraries and MTA data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Import turnstile data from the MTA website
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], keep_date_col=True))
    return pd.concat(dfs)   

# April-September 2019
week_nums = [190406,190413,190420,190427,190504,190511,190518,190525,190601,190608,190615,190622,190629,190706,190713,190720,190727,190803,190810,190817,190824,190831,190907,190914,190921,190928] 

turnstiles_df = get_data(week_nums)

## Data Cleaning

In [16]:
# strip whitespace in EXITS column
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS
205335,2019-09-27 21:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,21:00:00,5554,420
205334,2019-09-27 17:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,17:00:00,5554,420
205333,2019-09-27 13:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,13:00:00,5554,420
205332,2019-09-27 09:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,09:00:00,5554,420
205331,2019-09-27 05:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,05:00:00,5554,420


In [17]:
# Looks like the duplicates are caused by recovered AUD values (RECOVR AUD). 
# Since there's already a REGULAR entry for each, get rid of the duplicate entries.
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

# Drop the DESC Column.  To prevent errors in multiple run of cell, errors on drop is ignored
turnstiles_df = turnstiles_df.drop(["DESC"], axis=1, errors="ignore")

# Also drop all columns that are to do with individual turnstiles or station areas
# turnstiles_df = turnstiles_df.drop(["C/A","UNIT","SCP","LINENAME","DIVISION"], axis=1, errors="ignore")
turnstiles_df.head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS
205335,2019-09-27 21:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,21:00:00,5554,420
205334,2019-09-27 17:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,17:00:00,5554,420
205333,2019-09-27 13:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,13:00:00,5554,420
205332,2019-09-27 09:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,09:00:00,5554,420
205331,2019-09-27 05:00:00,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/27/2019,05:00:00,5554,420


In [18]:
# There are many unique times that don't fall on the regular bin times!
turnstiles_df["DATE_TIME"].dt.time.nunique()

79286

In [19]:
turnstiles_block = (turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION","DATE_TIME"],as_index=False)
                    [["ENTRIES","EXITS"]].first())
turnstiles_block

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,2019-03-30 00:00:00,6999064,2373568
1,A002,R051,02-00-00,59 ST,2019-03-30 04:00:00,6999084,2373576
2,A002,R051,02-00-00,59 ST,2019-03-30 08:00:00,6999107,2373622
3,A002,R051,02-00-00,59 ST,2019-03-30 12:00:00,6999214,2373710
4,A002,R051,02-00-00,59 ST,2019-03-30 16:00:00,6999451,2373781
...,...,...,...,...,...,...,...
5351033,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-09-27 05:00:00,5554,420
5351034,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-09-27 09:00:00,5554,420
5351035,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-09-27 13:00:00,5554,420
5351036,TRAM2,R469,00-05-01,RIT-ROOSEVELT,2019-09-27 17:00:00,5554,420


In [20]:
turnstiles_block[["PREV_DATE","PREV_ENTRIES", "PREV_EXITS"]] = (
    turnstiles_block.groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE_TIME","ENTRIES", "EXITS"]]
    .apply(lambda grp: grp.shift(1)))

turnstiles_block.head(10)



,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS
0,A002,R051,02-00-00,59 ST,2019-03-30 00:00:00,6999064,2373568,NaT,NaN,NaN
1,A002,R051,02-00-00,59 ST,2019-03-30 04:00:00,6999084,2373576,2019-03-30 00:00:00,6999064.0,2373568.0
2,A002,R051,02-00-00,59 ST,2019-03-30 08:00:00,6999107,2373622,2019-03-30 04:00:00,6999084.0,2373576.0
3,A002,R051,02-00-00,59 ST,2019-03-30 12:00:00,6999214,2373710,2019-03-30 08:00:00,6999107.0,2373622.0
4,A002,R051,02-00-00,59 ST,2019-03-30 16:00:00,6999451,2373781,2019-03-30 12:00:00,6999214.0,2373710.0
5,A002,R051,02-00-00,59 ST,2019-03-30 20:00:00,6999796,2373837,2019-03-30 16:00:00,6999451.0,2373781.0
6,A002,R051,02-00-00,59 ST,2019-03-31 00:00:00,6999957,2373867,2019-03-30 20:00:00,6999796.0,2373837.0
7,A002,R051,02-00-00,59 ST,2019-03-31 04:00:00,6999983,2373876,2019-03-31 00:00:00,6999957.0,2373867.0
8,A002,R051,02-00-00,59 ST,2019-03-31 08:00:00,6999998,2373900,2019-03-31 04:00:00,6999983.0,2373876.0
9,A002,R051,02-00-00,59 ST,2019-03-31 12:00:00,7000069,2373957,2019-03-31 08:00:00,6999998.0,2373900.0


In [21]:
turnstiles_block.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [22]:
# determine daily entries and exits based on prev days
def get_daily_entries(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        #print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

def get_daily_exits(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        #print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

In [23]:
turnstiles_block["4HR_ENTRIES"] = (turnstiles_block.apply(get_daily_entries, 
                                                            axis=1, max_counter=100000))
turnstiles_block["4HR_EXITS"] = (turnstiles_block.apply(get_daily_exits, 
                                                          axis=1, max_counter=100000))
turnstiles_block[turnstiles_block["STATION"]=="BAY RIDGE-95 ST"].head(10)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,4HR_ENTRIES,4HR_EXITS
630793,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-30 04:00:00,5079382,6909918,2019-03-30 00:00:00,5079377.0,6909875.0,5.0,43.0
630794,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-30 08:00:00,5079421,6909945,2019-03-30 04:00:00,5079382.0,6909918.0,39.0,27.0
630795,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-30 12:00:00,5079512,6910015,2019-03-30 08:00:00,5079421.0,6909945.0,91.0,70.0
630796,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-30 16:00:00,5079604,6910134,2019-03-30 12:00:00,5079512.0,6910015.0,92.0,119.0
630797,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-30 20:00:00,5079659,6910315,2019-03-30 16:00:00,5079604.0,6910134.0,55.0,181.0
630798,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-31 00:00:00,5079680,6910425,2019-03-30 20:00:00,5079659.0,6910315.0,21.0,110.0
630799,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-31 04:00:00,5079683,6910476,2019-03-31 00:00:00,5079680.0,6910425.0,3.0,51.0
630800,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-31 08:00:00,5079703,6910504,2019-03-31 04:00:00,5079683.0,6910476.0,20.0,28.0
630801,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-31 12:00:00,5079785,6910566,2019-03-31 08:00:00,5079703.0,6910504.0,82.0,62.0
630802,C027,R216,00-00-00,BAY RIDGE-95 ST,2019-03-31 16:00:00,5079851,6910673,2019-03-31 12:00:00,5079785.0,6910566.0,66.0,107.0


In [24]:
# Group odd times in the standard 4 hour time blocks

def time_block(time):
    if time in {22,23,0,1}: 
        time_block = "20:00 - 24:00"
    elif time in range(2,6):
        time_block = "00:00 - 4:00"
    elif time in range(6,10):
        time_block = "04:00 - 08:00"
    elif time in range(10,14):
        time_block = "08:00 - 12:00"
    elif time in range(14,18):
        time_block = "12:00 - 16:00"
    elif time in range(18,22):
        time_block = "16:00 - 20:00" 
    return time_block

In [25]:
# Apply time_block function to group data into 4 hour time blocks
turnstiles_block["BIN_HOUR"] = turnstiles_block["DATE_TIME"].dt.hour.apply(time_block)
turnstiles_block.head(10)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,4HR_ENTRIES,4HR_EXITS,BIN_HOUR
1,A002,R051,02-00-00,59 ST,2019-03-30 04:00:00,6999084,2373576,2019-03-30 00:00:00,6999064.0,2373568.0,20.0,8.0,00:00 - 4:00
2,A002,R051,02-00-00,59 ST,2019-03-30 08:00:00,6999107,2373622,2019-03-30 04:00:00,6999084.0,2373576.0,23.0,46.0,04:00 - 08:00
3,A002,R051,02-00-00,59 ST,2019-03-30 12:00:00,6999214,2373710,2019-03-30 08:00:00,6999107.0,2373622.0,107.0,88.0,08:00 - 12:00
4,A002,R051,02-00-00,59 ST,2019-03-30 16:00:00,6999451,2373781,2019-03-30 12:00:00,6999214.0,2373710.0,237.0,71.0,12:00 - 16:00
5,A002,R051,02-00-00,59 ST,2019-03-30 20:00:00,6999796,2373837,2019-03-30 16:00:00,6999451.0,2373781.0,345.0,56.0,16:00 - 20:00
6,A002,R051,02-00-00,59 ST,2019-03-31 00:00:00,6999957,2373867,2019-03-30 20:00:00,6999796.0,2373837.0,161.0,30.0,20:00 - 24:00
7,A002,R051,02-00-00,59 ST,2019-03-31 04:00:00,6999983,2373876,2019-03-31 00:00:00,6999957.0,2373867.0,26.0,9.0,00:00 - 4:00
8,A002,R051,02-00-00,59 ST,2019-03-31 08:00:00,6999998,2373900,2019-03-31 04:00:00,6999983.0,2373876.0,15.0,24.0,04:00 - 08:00
9,A002,R051,02-00-00,59 ST,2019-03-31 12:00:00,7000069,2373957,2019-03-31 08:00:00,6999998.0,2373900.0,71.0,57.0,08:00 - 12:00
10,A002,R051,02-00-00,59 ST,2019-03-31 16:00:00,7000220,2374010,2019-03-31 12:00:00,7000069.0,2373957.0,151.0,53.0,12:00 - 16:00


In [26]:
turnstiles_block.to_pickle('turnstiles_block.pkl')


In [ ]:
## Import Python libraries and MTA data

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Import turnstile data from the MTA website
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], keep_date_col=True))
    return pd.concat(dfs)   

# April-September 2019
week_nums = [190406,190413,190420,190427,190504,190511,190518,190525,190601,190608,190615,190622,190629,190706,190713,190720,190727,190803,190810,190817,190824,190831,190907,190914,190921,190928] 

turnstiles_df = get_data(week_nums)

## Data Cleaning

# strip whitespace in EXITS column
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.head()

# Looks like the duplicates are caused by recovered AUD values (RECOVR AUD). 
# Since there's already a REGULAR entry for each, get rid of the duplicate entries.
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

# Drop the DESC Column.  To prevent errors in multiple run of cell, errors on drop is ignored
turnstiles_df = turnstiles_df.drop(["DESC"], axis=1, errors="ignore")

# Also drop all columns that are to do with individual turnstiles or station areas
# turnstiles_df = turnstiles_df.drop(["C/A","UNIT","SCP","LINENAME","DIVISION"], axis=1, errors="ignore")
turnstiles_df.head()

# There are many unique times that don't fall on the regular bin times!
turnstiles_df["DATE_TIME"].dt.time.nunique()

turnstiles_block = (turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION","DATE_TIME"],as_index=False)
                    [["ENTRIES","EXITS"]].first())
turnstiles_block

turnstiles_block[["PREV_DATE","PREV_ENTRIES", "PREV_EXITS"]] = (
    turnstiles_block.groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE_TIME","ENTRIES", "EXITS"]]
    .apply(lambda grp: grp.shift(1)))

turnstiles_block.head(10)



turnstiles_block.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

# determine daily entries and exits based on prev days
def get_daily_entries(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        #print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

def get_daily_exits(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        #print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

turnstiles_block["4HR_ENTRIES"] = (turnstiles_block.apply(get_daily_entries, 
                                                            axis=1, max_counter=100000))
turnstiles_block["4HR_EXITS"] = (turnstiles_block.apply(get_daily_exits, 
                                                          axis=1, max_counter=100000))
turnstiles_block[turnstiles_block["STATION"]=="BAY RIDGE-95 ST"].head(10)

# Group odd times in the standard 4 hour time blocks

def time_block(time):
    if time in {22,23,0,1}: 
        time_block = "20:00 - 24:00"
    elif time in range(2,6):
        time_block = "00:00 - 4:00"
    elif time in range(6,10):
        time_block = "04:00 - 08:00"
    elif time in range(10,14):
        time_block = "08:00 - 12:00"
    elif time in range(14,18):
        time_block = "12:00 - 16:00"
    elif time in range(18,22):
        time_block = "16:00 - 20:00" 
    return time_block

# Apply time_block function to group data into 4 hour time blocks
turnstiles_block["BIN_HOUR"] = turnstiles_block["DATE_TIME"].dt.hour.apply(time_block)
turnstiles_block.head(10)

turnstiles_block.to_pickle('turnstiles_block.pkl')
